In [1]:
import autograd
import autograd.numpy as np
from autograd import grad
from autograd import checkpoint
from autograd.extend import primitive

import numpy as onp
from time import time

%load_ext memory_profiler

from builtins import range, list as ag_list, tuple as ag_tuple
import autograd.numpy as np
import autograd.numpy.random as npr
from autograd import grad
from autograd.scipy.misc import logsumexp
from os.path import dirname, join
from autograd.misc.optimizers import adam

In [2]:
def create_rnn_params(input_size, state_size, output_size,
                      param_scale=0.01, rs=npr.RandomState(0)):
    return {'change': rs.randn(input_size + state_size + 1, state_size) * param_scale,
            'predict': rs.randn(state_size + 1, output_size) * param_scale,
            'init hiddens': rs.randn(1, state_size) * param_scale,}

def sigmoid(x):
    return 0.5*(np.tanh(x) + 1.0)   # Output ranges from 0 to 1.

def hiddens_to_output_probs(theta, hiddens):
    output = concat_and_multiply(theta['predict'], hiddens)
    return output - logsumexp(output, axis=1, keepdims=True)

def concat_and_multiply(weights, *args):
    cat_state = np.hstack(args + (np.ones((args[0].shape[0], 1)),))
    return np.dot(cat_state, weights)

input_size = 64
state_size = 64
output_size = 64

batch_size = 64
seq_len = 512
num_checkpoints = 32

theta = create_rnn_params(input_size, state_size, output_size)

np.random.seed(0)
inputs = [np.random.randn(batch_size, input_size) for _ in range(seq_len)]

In [3]:
from autograd.differential_operators import binomial_checkpoint
from autograd.builtins import list as ag_list, tuple as ag_tuple

def rnn(theta, cell, x):    
    def update_rnn(x, cell):
        return np.tanh(concat_and_multiply(theta['change'], x, cell))

    return update_rnn(x, cell)    

In [4]:
def rnn_predict(params, inputs):
    num_sequences = inputs[0].shape[0]
    
    hidden_single = npr.RandomState(0).randn(1, state_size) * .01
    hidden = np.repeat(hidden_single, num_sequences, axis=0)  
    output = hiddens_to_output_probs(params, hidden)
    
    outputs = [output]

    for input in inputs:  # Iterate over time steps.
        hidden = rnn(params, hidden, input)
        output = hiddens_to_output_probs(params, hidden)
        
        outputs.append(output)
    
    return outputs

f = lambda theta: rnn_predict(theta, inputs)

In [5]:
f = lambda theta: rnn_predict(theta, inputs)
g = lambda theta: np.sum(sum(f(theta)))

x2 = g(theta)

start = time()
%memit g2 = grad(g)(theta)
end = time()

print("elapsed time: ", end - start)

peak memory: 212.46 MiB, increment: 108.35 MiB
elapsed time:  0.9083609580993652


In [6]:
loop = binomial_checkpoint(rnn, seq_len, num_checkpoints)

def rnn_predict(params, inputs):
    num_sequences = inputs[0].shape[0]
    
    hidden_single = npr.RandomState(0).randn(1, state_size) * .01
    hidden = np.repeat(hidden_single, num_sequences, axis=0)
    hiddens = loop(params, hidden, inputs)
    
    return [hiddens_to_output_probs(params, hidden) for hidden in hiddens]

In [7]:
f = lambda theta: rnn_predict(theta, inputs)
g = lambda theta: np.sum(sum(f(theta)))

x1 = g(theta)

start = time()
%memit g1 = grad(g)(theta)
end = time()

print("elapsed time: ", end - start)

peak memory: 169.31 MiB, increment: 2.27 MiB
elapsed time:  1.6155798435211182


In [8]:
# check correctness of gradients

print(x1 - x2)

for key in g1:
    print(key, np.sum((g1[key] - g2[key])**2))

0.0
change 3.4627515652098547e-26
predict 0.0
init hiddens 0.0
